In [4]:
import sys, os, re; sys.path = (["../src/", "../"] if re.match(r'^(\w\:\\)|(/)', os.getcwd()) else [])+ sys.path 

from typing import Any, Optional, List
# import asyncio
# import nest_asyncio
# nest_asyncio.apply()

import qubx
%qubxd dev 

%load_ext autoreload
%autoreload 2

from qubx import lookup, logger
from qubx.core.strategy import IStrategy, StrategyContext, TriggerEvent
from qubx.pandaz.utils import *

# - - - - - - - - - - - - - - - - - - - - - - - -
from qubx.backtester.ome import OrderManagementEngine
from qubx.trackers import PortfolioRebalancerTracker
from qubx.data.readers import CsvStorageDataReader, AsTimestampedRecords, AsQuotes, RestoreTicksFromOHLC
from qubx.core.basics import Deal, Instrument, Order, Position, Signal

 >  [dev] installing cython rebuilding hook
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# OME tests

In [5]:
r = CsvStorageDataReader('../tests/data/csv')

In [8]:
stream = r.read('BTCUSDT_ohlcv_M1', transform=RestoreTicksFromOHLC(trades=True))

In [14]:
from sortedcontainers import SortedDict

orders: SortedDict[float, Order] = SortedDict()
orders[12.0] = Order("10000", "MARKET", "BTCUSDT",  np.datetime64("2020-01-01"), 100, 12.0, "BUY", "PENDING", "gtc")
orders[13.0] = Order("10000", "MARKET", "BTCUSDT",  np.datetime64("2020-01-01"), 200, 13.0, "SELL", "PENDING", "gtc")

SortedDict({12.0: Order(id='10000', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2020-01-01'), quantity=100, price=12.0, side='BUY', status='PENDING', time_in_force='gtc', client_id=None, cost=0.0), 13.0: Order(id='10000', type='MARKET', symbol='BTCUSDT', time=numpy.datetime64('2020-01-01'), quantity=200, price=13.0, side='SELL', status='PENDING', time_in_force='gtc', client_id=None, cost=0.0)})


# <font color='red'>(TODO)</font> Attempt to backtest strategy 

In [ ]:
class TestStrategy(IStrategy):

    def on_start(self, ctx: StrategyContext):
        logger.info(f"Strategy started | {ctx.time}")

    def on_fit(self, ctx: 'StrategyContext', fit_time: str | pd.Timestamp, previous_fit_time: str | pd.Timestamp | None = None):
        logger.info(f"> Fit is called | fit_time: {fit_time} / prev: {previous_fit_time}")

    def on_event(self, ctx: StrategyContext, event: TriggerEvent) -> List[Signal] | None:
        logger.info(f"{event.time} -> {event}")
        # ohlcs = self.ohlcs('15Min')

        # symbols_to_close, symbols_to_open = [], []
        # for s, p in ctx.positions.items():
        #     if p.quantity != 0: symbols_to_close.append(s)
        #     else: symbols_to_open.append(s)

        # if not symbols_to_close: # first run just open half from all universe
        #     symbols_to_open = symbols_to_open[:len(symbols_to_open) // 2]

        # cap = self._tracker.estimate_capital_to_trade(symbols_to_close)
        # capital_per_symbol = np.clip(round(cap.capital / len(symbols_to_open)), 5, np.inf)

        # logger.info(f"\n>>> to close: {symbols_to_close}\n>>> to open: {symbols_to_open} | {capital_per_symbol} per symbol")

        # c_time = ctx.time()
        # to_open = pd.DataFrame({s: {c_time: capital_per_symbol / ohlcs[s].close.iloc[-1]} for s in symbols_to_open})
        # to_close = pd.DataFrame({s: {c_time: 0} for s in symbols_to_close})
        # signals = scols(to_close, to_open)

        # self.reporting(signals, cap)

        # # - process signals
        # if self.trading_allowed:
        #     self._tracker.process_signals(signals)
        # else:
        #     logger.warning("Trading is disabled - no postions will be changed")

        return None

    def ohlcs(self, timeframe:str) -> Dict[str, pd.DataFrame]:
        return {s.symbol: self.ctx.ohlc(s, timeframe).pd() for s in self.ctx.instruments}

    def on_stop(self, ctx: StrategyContext):
        logger.info(f"> test is stopped")

    def tracker(self, ctx: StrategyContext) -> PortfolioRebalancerTracker:
        return PortfolioRebalancerTracker(ctx, 1000, 0)

In [ ]:
ctx = StrategyContext(
    TestStrategy(), dict(
        capital_invested=100,
        trading_allowed=False
    ), 
    cc, cc, instruments=instruments,  # type: ignore
    md_subscription=dict(type='ohlc', timeframe='5Min', nback=100),
    trigger=dict(type='bar', delay='-5Sec', timeframe='1Min')
)
S = ctx.strategy